# Fraud in Mobile Money Transactions
PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world.

# Libraries

In [1]:
# allows us to ignore warnings, since they are just warnings 
import warnings
warnings.filterwarnings('ignore')

# allows us to print things in jupyter in a pretty way
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# import pandas to be able to use dataframes
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.6f' % x)

# import plotting library
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# allow inline plotting on notebook cells
%matplotlib inline
plt.switch_backend('agg')

# we'll get access to the numpy library
import numpy as np

# We'll use a label encoder to convert categorical feature to numerical values
from sklearn.preprocessing import LabelEncoder

# install using: pip install outlier_utils
# univariate anomaly detection
from outliers import smirnov_grubbs as grubbs

# Access to the localoutlierfactor for clustering
from sklearn.neighbors import LocalOutlierFactor

# allows us to implement a train test split strategy
from sklearn.model_selection import train_test_split

# access to the logistic regression class of Sklearn
from sklearn.linear_model import LogisticRegression

# access to the neural network model MLP Classifier
from sklearn.neural_network import MLPClassifier

# classification metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# for use with CAP report
from scipy import integrate

# feature power transformation functions
from scipy.stats import boxcox
from scipy.special import boxcox1p

# normality tests
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import probplot

# pip install imbalanced-learn (open anaconda prompt as admin)
# sudo pip install imbalanced-learn (macos/linux)
# pip install delayed (only do this if required)
# This module is used for handling imbalanced data
from imblearn.over_sampling import SMOTE

# Collections is a core python module. We wille use counter only for reporting
from collections import Counter




# Exploration

In [2]:
# https://www.kaggle.com/ntnu-testimon/paysim1/download
# download archive.zip file and place in data/ folder 
data_df = pd.read_csv('data/archive.zip')

In [3]:
data_df.shape

(6362620, 11)

In [4]:
# Let's fix some of the columns 
data_df = data_df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})

# Header
This is a sample of 1 row with headers explanation:

1,PAYMENT,1060.31,C429214117,1089.0,28.69,M1591654462,0.0,0.0,0,0

step - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

amount -
amount of the transaction in local currency.

nameOrig - customer who started the transaction

oldbalanceOrg - initial balance before the transaction

newbalanceOrig - new balance after the transaction

nameDest - customer who is the recipient of the transaction

oldbalanceDest - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

newbalanceDest - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

isFraud - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

In [5]:
# Let's have a quick view of the dataset
tmp_df = data_df.head(1).T
tmp_df.columns = ['sample']
tmp_df['dtypes'] = data_df.dtypes
tmp_df

,sample,dtypes
step,1,int64
type,PAYMENT,object
amount,9839.640000,float64
nameOrig,C1231006815,object
oldBalanceOrig,170136.000000,float64
newBalanceOrig,160296.360000,float64
nameDest,M1979787155,object
oldBalanceDest,0.000000,float64
newBalanceDest,0.000000,float64
isFraud,0,int64


In [6]:
# Do we have rows that have any null values?
data_df.isnull().values.any()

False

In [7]:
# Let's try to better understand our data

col = 'type'
series = data_df[col]
tmp_df = pd.DataFrame(series.value_counts())
tmp_df = tmp_df.T


tmp_df.plot.bar(title=col,cmap="viridis")
plt.gcf().axes[0].yaxis.get_major_formatter().set_scientific(False)
plt.show()

In [8]:
tmp_df = data_df['amount']
tmp_df.describe()

count    6362620.000000
mean      179861.903549
std       603858.231463
min            0.000000
25%        13389.570000
50%        74871.940000
75%       208721.477500
max     92445516.640000
Name: amount, dtype: float64

In [9]:
tmp_df.hist()

<AxesSubplot:title={'center':'type'}>

In [10]:
col = 'isFraud'
series = data_df[col]
tmp_df = pd.DataFrame(series.value_counts())
tmp_df = tmp_df.T


tmp_df.plot.bar(title=col,cmap="viridis")
plt.gcf().axes[0].yaxis.get_major_formatter().set_scientific(False)
plt.show()

In [11]:
# There is a large imbalance between the two classes
display(tmp_df)

,0,1
isFraud,6354407,8213


In [12]:
# Let's revisit the exploration but with respect to each payment type

In [ ]:
col = 'type'
tmp_s = data_df[col]
tmp_list = tmp_s.unique()

for i in tmp_list:
    printmd("***"+i+"***")
    
    tmp_df = data_df[data_df['type']==i]
    display(tmp_df.describe()[['amount']].T)
    
    # fraud
    display(pd.DataFrame(tmp_df['isFraud'].value_counts()))
    
    # merchant vs regular customer
    
    # origin
    
    orig = {'regular': tmp_df[tmp_df.nameOrig.str.contains('C')].drop_duplicates().shape[0], 
             'merchant': tmp_df[tmp_df.nameOrig.str.contains('M')].drop_duplicates().shape[0]}
    dest = {'regular': tmp_df[tmp_df.nameDest.str.contains('C')].drop_duplicates().shape[0], 
             'merchant': tmp_df[tmp_df.nameDest.str.contains('M')].drop_duplicates().shape[0]}
    
    txn = [orig, dest]
    
    txn_df = pd.DataFrame(txn)
    txn_df.index = ["orig","dest"]
    
    display(txn_df)
  

***PAYMENT***

,count,mean,std,min,25%,50%,75%,max
amount,2151495.000000,13057.604660,12556.450186,0.020000,4383.820000,9482.190000,17561.220000,238637.980000


,isFraud
0,2151495


Looks like most frauds occured in Transfer and cash out and with regular customers

In [ ]:
# Let's look at consistency of balance within transactions before and after

# this should result to zero for a balanced transaction
data_df['errorOrig'] = (data_df['newBalanceOrig']+data_df['amount'])-data_df['oldBalanceOrig']
data_df['errorDest'] = (data_df['oldBalanceDest']+data_df['amount'])-data_df['newBalanceDest']

# what is the percentage of transactions with non-balanced (non ZERO) transactions?
print("% of orig txns with non zero error balance: ",data_df[data_df['errorOrig']!=0]['errorOrig'].count()/data_df['errorOrig'].shape[0])
print("% of dest txns with non zero error balance: ",data_df[data_df['errorDest']!=0]['errorDest'].count()/data_df['errorDest'].shape[0])

# how does it look like broken down by payment types?
print()
print("% of orig txns w/ non zero error balance by payment type")
tmp_df = data_df[data_df['errorOrig']!=0].groupby(['type']).count()['errorOrig']/data_df.groupby(['type']).count()['errorOrig']
tmp_df = pd.DataFrame(tmp_df)
display(tmp_df)

print("% of dest txns w/ non zero error balance by payment type")
tmp_df = data_df[data_df['errorDest']!=0].groupby(['type']).count()['errorDest']/data_df.groupby(['type']).count()['errorDest']
tmp_df = pd.DataFrame(tmp_df)
display(tmp_df)

# what about by fraud indicator?
print("% of orig txns w/ non zero error balance by fraud ind")
tmp_df = data_df[data_df['errorOrig']!=0].groupby(['isFraud']).count()['errorOrig']/data_df.groupby(['isFraud']).count()['errorOrig']
tmp_df = pd.DataFrame(tmp_df)
display(tmp_df)

print("% of dest txns w/ non zero error balance by fraud ind")
tmp_df = data_df[data_df['errorDest']!=0].groupby(['isFraud']).count()['errorDest']/data_df.groupby(['isFraud']).count()['errorDest']
tmp_df = pd.DataFrame(tmp_df)
display(tmp_df)

It seems in general having non zero error balance between transactions does not indicate any fraud behavior except for destination accounts which has higher than normal of 64% identified as fraud

In [ ]:
# how many unique fraud customers can be found in Cash out and transfer type transactions?
data_df.groupby(['isFraud','type']).nunique()[['nameOrig','nameDest']]

In [ ]:
fraud_df = data_df[data_df['isFraud']==1]
notfraud_df = data_df[data_df['isFraud']==0]

In [ ]:
# Are there any relationships within the customers and merchants that conducted fraud?

fraudorig_set = set(fraud_df['nameOrig'].values) #convert series to array then set (unique) 
frauddest_set = set(fraud_df['nameDest'].values) #convert series to array then set (unique)
fraudcombined_set = fraudorig_set.union(frauddest_set) #combine both into one set

notfraudorig_set = set(notfraud_df['nameOrig'].values) #convert series to array then set (unique) 
notfrauddest_set = set(notfraud_df['nameDest'].values) #convert series to array then set (unique)
notfraudcombined_set = notfraudorig_set.union(notfrauddest_set) #combine both into one set

In [ ]:
print("Total unique customer count: ", len(fraudcombined_set.union(notfraudcombined_set)))
print("Customers involved in fraud: ",len(fraudcombined_set))
print("Customers NOT involved in fraud: ",len(notfraudcombined_set))
print("Originating Customers in Txns: ", len(fraudorig_set.union(notfraudorig_set)))
print("Destination Customers in Txns: ", len(frauddest_set.union(notfrauddest_set)))
print("Count of customers that originated in fraud transactions: ",len(fraudorig_set))
print("How many of those customers are regular customers? ",len([i for i in fraudorig_set if 'C' in i]))
print("How many of those customers are merchant customers? ", len([i for i in fraudorig_set if 'M' in i]))
print("Count of customers that is the destination in fraud transactions: ",len(frauddest_set))
print("How many of those customers are regular customers? ",len([i for i in frauddest_set if 'C' in i]))
print("How many of those customers are merchant customers? ", len([i for i in frauddest_set if 'M' in i]))
print("how many Fraud customers showed up in BOTH orig and dest? ", len(frauddest_set.intersection(fraudorig_set)))


In [ ]:
print("Did any fraud orig customers show up in legitimate transactions? ")
print("How many transactions as originating customers? ", data_df[(data_df['nameOrig'].isin(fraudorig_set)) & (data_df['isFraud']==0)].shape[0])
print("How many unique ids showed up as originating customers: ", len(fraudorig_set.intersection(notfraudorig_set)))
print("How many transactions as destination customers? ", data_df[(data_df['nameDest'].isin(fraudorig_set)) & (data_df['isFraud']==0)].shape[0])
print("How many unique ids showed up as destination customers: ", len(fraudorig_set.intersection(notfrauddest_set)))
print("")
print("Did any fraud dest customers show up in legitimate transactions? ")
print("How many transactions as originating customers? ", data_df[(data_df['nameOrig'].isin(frauddest_set)) & (data_df['isFraud']==0)].shape[0])
print("How many unique ids showed up as originating customers: ", len(frauddest_set.intersection(notfraudorig_set)))
print("How many transactions as destination customers? ", data_df[(data_df['nameDest'].isin(frauddest_set)) & (data_df['isFraud']==0)].shape[0])
print("How many unique ids showed up as destination customers: ", len(frauddest_set.intersection(notfrauddest_set)))


In [ ]:
tmp_df = pd.DataFrame()
tmp_df['fraud'] = fraud_df.amount.describe().T
tmp_df['notfraud'] = notfraud_df.amount.describe().T
display(tmp_df.T)

# Preprocessing

In [ ]:
# do we still have any null values?
data_df.isnull().values.any()

In [ ]:
enc = LabelEncoder()
data_df['type_num'] = enc.fit_transform(data_df['type'])
data_df['type_num'] = data_df['type_num'].apply(lambda x: np.int64(x))

In [ ]:
# Let's remove the customer names since we won't be needing them for training

training_df = data_df.drop(['nameOrig','nameDest','isFlaggedFraud','type','step'], axis=1)


In [ ]:
training_df.head()

In [ ]:
training_df.dtypes

# Anomaly Detection
## via Clustering

Unsupervised Outlier Detection using Local Outlier Factor (LOF)

The anomaly score of each sample is called Local Outlier Factor. It measures the local deviation of density of a given sample with respect to its neighbors. It is local in that the anomaly score depends on how isolated the object is with respect to the surrounding neighborhood. More precisely, locality is given by k-nearest neighbors, whose distance is used to estimate the local density. By comparing the local density of a sample to the local densities of its neighbors, one can identify samples that have a substantially lower density than their neighbors. These are considered outliers.

In [ ]:
y = training_df.isFraud
X = training_df.drop(['isFraud'],axis=1)

In [ ]:
fgsdfgsdfgsdfg

In [ ]:
%%time

lof_model = LocalOutlierFactor(n_neighbors=2)
y_pred = lof_model.fit_predict(X)




In [ ]:
# lof works by tagging inliers with 1 and outliers by -1
y_pred

In [ ]:
# Let's see how many outliers our unsupervised model detected (outliers are tagged as -1)

tmp_df = pd.DataFrame(y_pred)
tmp_df.value_counts()

negative_outlier_factor_ is The opposite LOF of the training samples. The higher, the more normal. Inliers tend to have a LOF score close to 1 (negative_outlier_factor_ close to -1), while outliers tend to have a larger LOF score.


In [ ]:
lof_model.negative_outlier_factor_

You can think of the negative_outlier_factor_ as our indicator on how 'far' the outlier is from the neighborhood densities.

This demonstrates that we can use an unsupervised modeling approach to fraud detection in the event that we do not have yet substantial data on caught fraudsters. It is a good starting point to build our way towards building supervised models.

## via Grubb's Test

Grubbs’ Test is used to identify the presence of outliers in a dataset. To use this test, a dataset should be approximately normally distributed and have at least 7 observations. Grubbs's test is based on the assumption of **normality**. That is, one should first verify that the data can be reasonably approximated by a normal distribution before applying the Grubbs test.

https://en.wikipedia.org/wiki/Grubbs%27s_test

**For purposes of demonstration we will only use ONE FEATURE to show case the tests**

In [ ]:
# we will look at two classes of techniques for checking whether a sample of data is Gaussian:

# Graphical Methods. These are methods for plotting the data and qualitatively evaluating whether the data looks Gaussian.
# Statistical Tests. These are methods that calculate statistics on the data and quantify how likely it is that
# the data was drawn from a Gaussian distribution. Methods of this type are often called normality tests.

# via Graphical - We assume that plotting our data set would result to what would look like a 'normal' distribution

# histogram - is an approximate representation of the distribution of numerical data

series_tmp = training_df['amount'].copy()
display("Before")
series_tmp.hist()
plt.show()
# you can see here that our distribution skews to the left which is different to the bellcurve of a normal/gaussian distribution

# To address this, we use a power transformation and try to make our dataset look normal/gaussian
# The Boxcox function accepts a lambda parameter that dictates what transformation to do
# In this case a lambda value of 0 means it will do a log transformation to all of the data set
boxcox_series_tmp = pd.Series(boxcox1p(series_tmp,0))
display("After")
series_tmp.hist()
plt.show()

# You can see here that our data set is now more 'gaussian' like

In [ ]:
# The Q-Q plot, or quantile-quantile plot, is a graphical tool to help us assess if
# a set of data plausibly came from some theoretical distribution such as a Normal or exponential. 

display("Before")
probplot(series_tmp, dist="norm", plot=plt)
plt.show()

# You can see that our data set does not completely fit our plot, so we apply the same power transformation

display("After")

probplot(boxcox_series_tmp, dist="norm", plot=plt)
plt.show()

# We plot the data set again and see a minor visual improvements to the qq plot


In [ ]:
# The next tests validate by generating a statistic called p value 
# Technically this is called the null hypothesis, or H0. A threshold level is chosen called alpha, 
# typically 5% (or 0.05), that is used to interpret the p-value.

# In the SciPy implementation of these tests, you can interpret the p value as follows.

# p <= alpha: reject H0, not normal.
# p > alpha: fail to reject H0, normal.



print("Using Shapiro Normality Test")
print("Before: ")
print(shapiro(series_tmp))

print("After")
print(shapiro(boxcox_series_tmp))

# The Shapiro-Wilk test evaluates a data sample and quantifies how likely it is that the data was drawn from a
# Gaussian distribution, named for Samuel Shapiro and Martin Wilk.

print("")

print("Using D’Agostino’s K^2 Normality Test")
print("Before: ")
print(normaltest(series_tmp))

print("After")
print(normaltest(boxcox_series_tmp))


# The normal test AKA D’Agostino’s K^2 test calculates summary statistics from the data, namely kurtosis and skewness,
# to determine if the data distribution departs from the normal distribution, named for Ralph D’Agostino.

Our statistical tests are telling us that our data set it not normally distributed, because of the resulting pvalue below .05 threshold despite our power transformations. On the other hand our graphical tests tells us that our data after the transformations are a bit more gaussian /normal like. No tests are perfect as well as no data is in reality 'completely' normally distributed. We can use this as decision point to move forward and treat our data set as gaussian like.

In [ ]:
# test for normality first
# transform to normal / guassian distribution to fit  if normality test fails
# run grubb's test



print("Our transformed data set length count: ")
print(len(boxcox_series_tmp))
grubbs_result = grubbs.test(boxcox_series_tmp, alpha=0.05)

print("Our Grubbs test result length count: ")
print(len(grubbs_result))

print("Grubs test works by returning the dataset and removing the anomalies, you can see that we returned missing 1, which means we found 1 anomaly")

In [ ]:
# Let's look at what index was dropped by the test 
set(list(boxcox_series_tmp.index)).difference(set(list(grubbs_result.index)))

In [ ]:
# what value did the test consider as outlier?
boxcox_series_tmp[2736447]

# SMOTE for oversampling
SMOTE stands for Synthetic Minority Oversampling Technique. This is a statistical technique for increasing the number of cases in your dataset in a balanced way.

Since our fraud dataset is imbalanced (low fraud cases), we will use oversampling to try to 'balance' the classes. This will results to more robust trained models that would be less prone to under or overfitting due to under or over representation.

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
# summarize the new class distribution
counter = Counter(y)
print(counter)

# Logistic Regression

As our base model we will use a simple logistic regression as our starting point for developing our predictive model.

In [ ]:
# here we do a few things: 1) separate our data set into 30% and 70% splits
# 70% will be used for as training data set
# 30% will be used to test the resulting model
# X represents our features
# y reprsents our target variable (1 is most likely to apply to the company after the training)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X.columns 
# we keep the column names to be used later

# Base model using all variables
logreg_model = LogisticRegression(random_state=0)

# Let's input our training data set and fit our model
logreg_model.fit(X_train, y_train)

# Quick review of feature importance
Using our trained logistic regression model, we will extract the computed coefficients and use that as basis for reporting the the effect of each feature to with respect to predicting the classification. A positive indicates the likelihood to result to a 1 while a negative value a 0. The value indicates the degree of impact to the classification.

In [ ]:
importance = logreg_model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d - %s, Score: %.6f' % (i,columns[i],v))

# plot feature importance
plt.bar([columns[x] for x in range(len(importance))], importance)
plt.xticks(rotation=90)
plt.show()

# Let's Predict

In [ ]:
# predict against X_test
y_pred = logreg_model.predict(X_test)
y_pred

In [ ]:
# Let's save our results in a variable for later use
y_pred_lr = y_pred

# How did our logistic regression perform?

print("accuracy: ",accuracy_score(y_test,y_pred_lr))
print("precision_score: ",precision_score(y_test,y_pred_lr))
print("recall_score: ",recall_score(y_test,y_pred_lr))
print("confusion matrix: ")
confusion_matrix(y_test,y_pred_lr)

In [ ]:
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
lr_probs = y_pred_lr
# keep probabilities for the positive outcome only
#lr_probs = lr_probs[:, 1]

# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic Regression')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

# Neural Network

As our base model we will use a simple logistic regression as our starting point for developing our predictive model.

In [ ]:
%%time 

# Class MLPClassifier implements a multi-layer perceptron (MLP) algorithm that trains using Backpropagation.



nn_model = MLPClassifier(random_state=0)

nn_model.fit(X_train, y_train)


In [ ]:
# predict against X_test
y_pred = nn_model.predict(X_test)
y_pred

In [ ]:
# Let's save our results in a variable for later use
y_pred_nn = y_pred

# How did our neural network model perform?

print("accuracy: ",accuracy_score(y_test,y_pred_nn))
print("precision_score: ",precision_score(y_test,y_pred_nn))
print("recall_score: ",recall_score(y_test,y_pred_nn))
print("confusion matrix: ")
confusion_matrix(y_test,y_pred_nn)

In [ ]:
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
lr_probs = y_pred_nn

# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Neural Network: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Neural Network')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

# Revised Neural Network
Let's remodel the Neural Network but this time, let's filter out oldBalanceDest, newBalanceDest, and type_num since based on the feature importance review, these features contributed least to the predicting factor of the model.

In [ ]:
# Let's remove the features and create our revised train test dataset
revised_X = X.drop(['oldBalanceDest','newBalanceDest','type_num'], axis=1)
revised_X_train, revised_X_test, y_train, y_test = train_test_split(revised_X, y, test_size=0.3, random_state=0)

In [ ]:
%%time 

# Class MLPClassifier implements a multi-layer perceptron (MLP) algorithm that trains using Backpropagation.



revised_nn_model = MLPClassifier(random_state=0)

revised_nn_model.fit(revised_X_train, y_train)


In [ ]:
# predict against X_test
y_pred = revised_nn_model.predict(revised_X_test)
y_pred

In [ ]:
# Let's save our results in a variable for later use
y_pred_revised_nn = y_pred

# How did our neural network model perform?

print("accuracy: ",accuracy_score(y_test,y_pred_revised_nn))
print("precision_score: ",precision_score(y_test,y_pred_revised_nn))
print("recall_score: ",recall_score(y_test,y_pred_revised_nn))
print("confusion matrix: ")
confusion_matrix(y_test,y_pred_revised_nn)

In [ ]:
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
lr_probs = y_pred_revised_nn

# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Neural Network: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Revised Neural Network')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

# Let's summarize our findings:

In [ ]:
tmp_list = []
tmp_dict = {"accuracy":accuracy_score(y_test,y_pred_lr), "precision":precision_score(y_test,y_pred_lr), "recall":recall_score(y_test,y_pred_lr),"roc auc": roc_auc_score(y_test,y_pred_lr)}
tmp_list.append(tmp_dict)
tmp_dict = {"accuracy":accuracy_score(y_test,y_pred_nn), "precision":precision_score(y_test,y_pred_nn), "recall":recall_score(y_test,y_pred_nn),"roc auc":roc_auc_score(y_test,y_pred_nn)}
tmp_list.append(tmp_dict)
tmp_dict = {"accuracy":accuracy_score(y_test,y_pred_revised_nn), "precision":precision_score(y_test,y_pred_revised_nn), "recall":recall_score(y_test,y_pred_revised_nn),"roc auc":roc_auc_score(y_test,y_pred_revised_nn)}
tmp_list.append(tmp_dict)

scores_df = pd.DataFrame(tmp_list)
scores_df.index = ['LogisticRegression','NeuralNetworks','RevisedNeuralNetworks']
scores_df

## How do we choose our champion model? IT DEPENDS

Fraud use cases are a tricky challenge to tackle given its nature of being inherently 'hidden'. It always implies that the dataset would always be imbalanced and that we would never have the complete picture of all the possible fraud indicators and that everytime we catch fraudsters they will inevitably change behavior to find other fraudulent techniques that have not yet been detected.

Its important to be anchored on the objective of the business and how it is affected by fraud. Depending on  the type of industry, a business may be affected by the cost of a fraud transactions, the cost of investigating a fraud case, the cost to entity's reputation, or the cost to the customer experience. Looking at all these factors will help one decide which model is based used that greatly cater's to the business need.

Given our curret use case, we **recommend the 2nd model (In this case, Neural Network)** which has a high precision as we have a high volume transactional business which means when investigating fraudulent activities, we want to limit the capture of false positives to control the cost of investigating fraud (which is normally high) and to minimize the impact to our customers (its annoying to be accused of fraud if you are actually a legit customer and have your transactions frozen).

## Cumulative Accuracy Profile (CAP) or Lorenz Curve

CAP popularly called the ‘Cumulative Accuracy Profile’ is used in the performance evaluation of the classification model. It helps us to understand and conclude about the robustness of the classification model. 

In [ ]:


def capcurve(y_values, y_preds_proba):
    num_pos_obs = np.sum(y_values)
    num_count = len(y_values)
    rate_pos_obs = float(num_pos_obs) / float(num_count)
    ideal = pd.DataFrame({'x':[0,rate_pos_obs,1],'y':[0,1,1]})
    xx = np.arange(num_count) / float(num_count - 1)
    
    y_cap = np.c_[y_values,y_preds_proba]
    y_cap_df_s = pd.DataFrame(data=y_cap)
    y_cap_df_s = y_cap_df_s.sort_values([1], ascending=False).reset_index(level = y_cap_df_s.index.names, drop=True)
    
    #print(y_cap_df_s.head(20))
    
    yy = np.cumsum(y_cap_df_s[0]) / float(num_pos_obs)
    yy = np.append([0], yy[0:num_count-1]) #add the first curve point (0,0) : for xx=0 we have yy=0
    
    percent = 0.5
    row_index = int(np.trunc(num_count * percent))
    
    val_y1 = yy[row_index]
    val_y2 = yy[row_index+1]
    if val_y1 == val_y2:
        val = val_y1*1.0
    else:
        val_x1 = xx[row_index]
        val_x2 = xx[row_index+1]
        val = val_y1 + ((val_x2 - percent)/(val_x2 - val_x1))*(val_y2 - val_y1)
    
    sigma_ideal = 1 * xx[num_pos_obs - 1 ] / 2 + (xx[num_count - 1] - xx[num_pos_obs]) * 1
    sigma_model = integrate.simps(yy,xx)
    sigma_random = integrate.simps(xx,xx)
    
    ar_value = (sigma_model - sigma_random) / (sigma_ideal - sigma_random)
    
    fig = plt.figure(figsize=(10, 6))
    ax = fig.subplots(nrows = 1, ncols = 1)

    
    ax.plot(ideal['x'],ideal['y'], color='grey', label='Perfect Model')
    ax.plot(xx,yy, color='red', label='Revised Neural Network Model')
    ax.plot(xx,xx, color='blue', label='Random Model')
    ax.plot([percent, percent], [0.0, val], color='green', linestyle='--', linewidth=1)
    ax.plot([0, percent], [val, val], color='green', linestyle='--', linewidth=1, label=str(val*100)+'% of positive obs at '+str(percent*100)+'%')
    
    plt.xlim(0, 1.02)
    plt.ylim(0, 1.25)
    plt.title("CAP Curve - a_r value ="+str(ar_value))
    plt.xlabel('% of the data')
    plt.ylabel('% of positive obs')
    plt.legend()
    plt.show()

    
# Let's store the pred probability for use later when we call the CAP function
revised_y_pred_proba = revised_nn_model.predict_proba(X=revised_X_test)

# Let's cal lthe cap curve function. What does this is pass the y_test (which contains our target classification
# as well as the predicted probabilities. The intention is to chart our CAP and see what is the optimal approach to 
# acting on our scored transaction list 
capcurve(y_test,revised_y_pred_proba[:,1])

The chart tells us that if we only investigate the top 50% of the scored transaction list, we will already get 97.56% of all fraudulent cases.

In [ ]:
# Let's create our white list. This white list will be the list of transactions that we will spend resources to investigate
# as they are the most probable fraud cases based on our revised neural network model

save_whitelist_df = revised_X_test.copy() # Let's create a dataframe of the test dataset
save_whitelist_df['y_pred_proba'] = revised_y_pred_proba[:,1] # store the probability to be a fraud case
save_whitelist_df['y_pred'] = y_pred_nn # add the predictions of our revised neural network
save_whitelist_df = save_whitelist_df.sort_values('y_pred_proba',ascending=False) # sort with highest to lowest probability

In [ ]:
# Let's get our cut off and filter to the final list and save to a CSV File

percentage = .49 # Let's use 49% since based on the chart we already hit the optimal cutoff before 50%
cutoff = int(save_whitelist_df.shape[0]*percentage) # will get the absolute count of 49% of total rows
print("Our cutoff value is ", cutoff)
save_whitelist_df = save_whitelist_df[0:cutoff] # we filter until the cutoff
save_whitelist_df.to_csv("fraud_revised_nn.csv") # Filter only to the predcted and save our CSV file
print("We predict ",save_whitelist_df['y_pred'].sum(), " fraud cases out of ", revised_X_test.shape[0], " transactions.")
save_whitelist_df.head()


## Dataset to try
- https://www.kaggle.com/c/ieee-fraud-detection